# Numerical dataset analysis

In this Jupyter notebook we analyze the distribution of eight scenes available in the LASIESTA dataset.
More precisely, we look at the

- overall distribution of background / foreground pixels per scene
- distribution of background / foreground pixels within certain pixel processes

In [9]:
from segmentizer import Segmentizer
from segmentizer.data_loader import LASIESTADataLoader
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from tqdm import tqdm_notebook, tnrange
from IPython.display import clear_output
import seaborn as sns
import pandas as pd
import numpy as np
from multiprocessing import Pool

## Overall histogram analysis

Pixel values can either belong to a foreground or background region.
To gather more insights we have to analyze the distribution of pixel values of each scene.
If we are able to show that foreground pixels tend to have a different mean / std than background pixels, we are in a good position to classify new pixels using a GMM.

We now "add up" all frames from a scene and compute
- mean / std of all foreground pixels
- mean / std of all background pixels
- mean / std of all pixels

In [3]:
def load_data(db_name):
    
    total_red = []
    total_green = []
    total_blue = []
    
    background_red = []
    background_green = []
    background_blue = []
    
    foreground_red = []
    foreground_green = []
    foreground_blue = []
    
    stats = dict()
    
    data_loader = LASIESTADataLoader('../datasets/'+db_name, '../datasets/'+db_name+'-GT', db_name)
    
    print("Start loading dataset " + db_name)
    
    for original_frame, label_frame in tqdm_notebook(data_loader, total=data_loader.get_nr_of_frames()):
        for i in range(original_frame.shape[0]):
            for j in range(original_frame.shape[1]):
                total_red.append(original_frame[i,j,0])
                total_green.append(original_frame[i,j,1])
                total_blue.append(original_frame[i,j,2])
          
                # Is background pixel
                if np.array_equal(label_frame[i,j], [0,0,0]):
                    background_red.append(original_frame[i,j,0])
                    background_green.append(original_frame[i,j,1])
                    background_blue.append(original_frame[i,j,2])  
                    
                # Is foreground pixel
                if np.array_equal(label_frame[i,j], [0,0,0]) is False and np.array_equal(label_frame[i,j], [128,128,128]) is False:
                    foreground_red.append(original_frame[i,j,0])
                    foreground_green.append(original_frame[i,j,1])
                    foreground_blue.append(original_frame[i,j,2])    
                
    stats['total_mean_red'] = np.mean(total_red)
    stats['total_mean_green'] = np.mean(total_green)
    stats['total_mean_blue'] = np.mean(total_blue)
    
    stats['total_std_red'] = np.std(total_red)
    stats['total_std_green'] = np.std(total_green)
    stats['total_std_blue'] = np.std(total_blue)
    
    stats['total_min_red'] = np.min(total_red)
    stats['total_min_green'] = np.min(total_green)
    stats['total_min_blue'] = np.min(total_blue)
    
    stats['total_max_red'] = np.max(total_red)
    stats['total_max_green'] = np.max(total_green)
    stats['total_max_blue'] = np.max(total_blue)
    
    stats['total_percentile_25_red'] = np.percentile(total_red, 25)
    stats['total_percentile_25_green'] = np.percentile(total_green, 25)
    stats['total_percentile_25_blue'] = np.percentile(total_blue, 25)
    
    stats['total_percentile_50_red'] = np.percentile(total_red, 50)
    stats['total_percentile_50_green'] = np.percentile(total_green, 50)
    stats['total_percentile_50_blue'] = np.percentile(total_blue, 50)
    
    stats['total_percentile_75_red'] = np.percentile(total_red, 75)
    stats['total_percentile_75_green'] = np.percentile(total_green, 75)
    stats['total_percentile_75_blue'] = np.percentile(total_blue, 75)
    
    stats['background_mean_red'] = np.mean(background_red)
    stats['background_mean_green'] = np.mean(background_green)
    stats['background_mean_blue'] = np.mean(background_blue)
    
    stats['background_std_red'] = np.std(background_red)
    stats['background_std_green'] = np.std(background_green)
    stats['background_std_blue'] = np.std(background_blue)
    
    stats['background_min_red'] = np.min(background_red)
    stats['background_min_green'] = np.min(background_green)
    stats['background_min_blue'] = np.min(background_blue)
    
    stats['background_max_red'] = np.max(background_red)
    stats['background_max_green'] = np.max(background_green)
    stats['background_max_blue'] = np.max(background_blue)
    
    stats['background_percentile_25_red'] = np.percentile(background_red, 25)
    stats['background_percentile_25_green'] = np.percentile(background_green, 25)
    stats['background_percentile_25_blue'] = np.percentile(background_blue, 25)
    
    stats['background_percentile_50_red'] = np.percentile(background_red, 50)
    stats['background_percentile_50_green'] = np.percentile(background_green, 50)
    stats['background_percentile_50_blue'] = np.percentile(background_blue, 50)
    
    stats['background_percentile_75_red'] = np.percentile(background_red, 75)
    stats['background_percentile_75_green'] = np.percentile(background_green, 75)
    stats['background_percentile_75_blue'] = np.percentile(background_blue, 75)
    
    stats['foreground_mean_red'] = np.mean(foreground_red)
    stats['foreground_mean_green'] = np.mean(foreground_green)
    stats['foreground_mean_blue'] = np.mean(foreground_blue)
    
    stats['foreground_std_red'] = np.std(foreground_red)
    stats['foreground_std_green'] = np.std(foreground_green)
    stats['foreground_std_blue'] = np.std(foreground_blue)
    
    stats['foreground_min_red'] = np.min(foreground_red)
    stats['foreground_min_green'] = np.min(foreground_green)
    stats['foreground_min_blue'] = np.min(foreground_blue)
    
    stats['foreground_max_red'] = np.max(foreground_red)
    stats['foreground_max_green'] = np.max(foreground_green)
    stats['foreground_max_blue'] = np.max(foreground_blue)
    
    stats['foreground_percentile_25_red'] = np.percentile(foreground_red, 25)
    stats['foreground_percentile_25_green'] = np.percentile(foreground_green, 25)
    stats['foreground_percentile_25_blue'] = np.percentile(foreground_blue, 25)
    
    stats['foreground_percentile_50_red'] = np.percentile(foreground_red, 50)
    stats['foreground_percentile_50_green'] = np.percentile(foreground_green, 50)
    stats['foreground_percentile_50_blue'] = np.percentile(foreground_blue, 50)
    
    stats['foreground_percentile_75_red'] = np.percentile(foreground_red, 75)
    stats['foreground_percentile_75_green'] = np.percentile(foreground_green, 75)
    stats['foreground_percentile_75_blue'] = np.percentile(foreground_blue, 75)
    
    print("Dataset " + db_name + ' loaded successfully')
    
    return stats
        

In [4]:
def print_overall_stats(stats_dataset):
    print('########## ' + dataset + ' ########## \n')
    print('Mean Total (Red): ' + str(stats_dataset['total_mean_red']))
    print('Std Total (Red): ' + str(stats_dataset['total_std_red']))
    print('Min Total (Red): ' + str(stats_dataset['total_min_red']))
    print('Max Total (Red): ' + str(stats_dataset['total_max_red']))
    print('25% Total (Red): ' + str(stats_dataset['total_percentile_25_red']))
    print('50% Total (Red): ' + str(stats_dataset['total_percentile_50_red']))
    print('75% Total (Red): ' + str(stats_dataset['total_percentile_75_red']))
    print()
    print('Mean Background (Red): ' + str(stats_dataset['background_mean_red']))
    print('Std Background (Red): ' + str(stats_dataset['background_std_red']))
    print('Min Background (Red): ' + str(stats_dataset['background_min_red']))
    print('Max Background (Red): ' + str(stats_dataset['background_max_red']))
    print('25% Background (Red): ' + str(stats_dataset['background_percentile_25_red']))
    print('50% Background (Red): ' + str(stats_dataset['background_percentile_50_red']))
    print('75% Background (Red): ' + str(stats_dataset['background_percentile_75_red']))
    print()
    print('Mean Foreground (Red): ' + str(stats_dataset['foreground_mean_red']))
    print('Std Foreground (Red): ' + str(stats_dataset['foreground_std_red']))
    print('Min Foreground (Red): ' + str(stats_dataset['foreground_min_red']))
    print('Max Foreground (Red): ' + str(stats_dataset['foreground_max_red']))
    print('25% Foreground (Red): ' + str(stats_dataset['foreground_percentile_25_red']))
    print('50% Foreground (Red): ' + str(stats_dataset['foreground_percentile_50_red']))
    print('75% Foreground (Red): ' + str(stats_dataset['foreground_percentile_75_red']))
    print()
    print('Mean Total (Green): ' + str(stats_dataset['total_mean_green']))
    print('Std Total (Green): ' + str(stats_dataset['total_std_green']))
    print('Min Total (Green): ' + str(stats_dataset['total_min_green']))
    print('Max Total (Green): ' + str(stats_dataset['total_max_green']))
    print('25% Total (Green): ' + str(stats_dataset['total_percentile_25_green']))
    print('50% Total (Green): ' + str(stats_dataset['total_percentile_50_green']))
    print('75% Total (Green): ' + str(stats_dataset['total_percentile_75_green']))
    print()
    print('Mean Background (Green): ' + str(stats_dataset['background_mean_green']))
    print('Std Background (Green): ' + str(stats_dataset['background_std_green']))
    print('Min Background (Green): ' + str(stats_dataset['background_min_green']))
    print('Max Background (Green): ' + str(stats_dataset['background_max_green']))
    print('25% Background (Green): ' + str(stats_dataset['background_percentile_25_green']))
    print('50% Background (Green): ' + str(stats_dataset['background_percentile_50_green']))
    print('75% Background (Green): ' + str(stats_dataset['background_percentile_75_green']))
    print()
    print('Mean Foreground (Green): ' + str(stats_dataset['foreground_mean_green']))
    print('Std Foreground (Green): ' + str(stats_dataset['foreground_std_green']))
    print('Min Foreground (Green): ' + str(stats_dataset['foreground_min_green']))
    print('Max Foreground (Green): ' + str(stats_dataset['foreground_max_green']))
    print('25% Foreground (Green): ' + str(stats_dataset['foreground_percentile_25_green']))
    print('50% Foreground (Green): ' + str(stats_dataset['foreground_percentile_50_green']))
    print('75% Foreground (Green): ' + str(stats_dataset['foreground_percentile_75_green']))
    print()
    print('Mean Total (Blue): ' + str(stats_dataset['total_mean_blue']))
    print('Std Total (Blue): ' + str(stats_dataset['total_std_blue']))
    print('Min Total (Blue): ' + str(stats_dataset['total_min_blue']))
    print('Max Total (Blue): ' + str(stats_dataset['total_max_blue']))
    print('25% Total (Blue): ' + str(stats_dataset['total_percentile_25_blue']))
    print('50% Total (Blue): ' + str(stats_dataset['total_percentile_50_blue']))
    print('75% Total (Blue): ' + str(stats_dataset['total_percentile_75_blue']))
    print()
    print('Mean Background (Blue): ' + str(stats_dataset['background_mean_blue']))
    print('Std Background (Blue): ' + str(stats_dataset['background_std_blue']))
    print('Min Background (Blue): ' + str(stats_dataset['background_min_blue']))
    print('Max Background (Blue): ' + str(stats_dataset['background_max_blue']))
    print('25% Background (Blue): ' + str(stats_dataset['background_percentile_25_blue']))
    print('50% Background (Blue): ' + str(stats_dataset['background_percentile_50_blue']))
    print('75% Background (Blue): ' + str(stats_dataset['background_percentile_75_blue']))
    print()
    print('Mean Foreground (Blue): ' + str(stats_dataset['foreground_mean_blue']))
    print('Std Foreground (Blue): ' + str(stats_dataset['foreground_std_blue']))
    print('Min Foreground (Blue): ' + str(stats_dataset['foreground_min_blue']))
    print('Max Foreground (Blue): ' + str(stats_dataset['foreground_max_blue']))
    print('25% Foreground (Blue): ' + str(stats_dataset['foreground_percentile_25_blue']))
    print('50% Foreground (Blue): ' + str(stats_dataset['foreground_percentile_50_blue']))
    print('75% Foreground (Blue): ' + str(stats_dataset['foreground_percentile_75_blue']))
    print()

In [5]:
datasets = [
    'I_BS_01',
    'I_CA_01',
    'I_IL_01',
    'I_MB_01',
    'I_MC_01',
    'I_OC_01',
    'I_SI_01',
    'I_SM_01'
]

stats_overall = dict()

for dataset in datasets:
    stats_overall[dataset] = load_data(dataset)
    print_overall_stats(stats_overall[dataset])

Start loading dataset I_BS_01


HBox(children=(IntProgress(value=0, max=275), HTML(value='')))


Dataset I_BS_01 loaded successfully
########## I_BS_01 ########## 

Mean Total (Red): 120.271118178
Std Total (Red): 42.5980364676
Min Total (Red): 0
Max Total (Red): 255
25% Total (Red): 103.0
50% Total (Red): 124.0
75% Total (Red): 148.0

Mean Background (Red): 121.709301625
Std Background (Red): 41.3602672181
Min Background (Red): 1
Max Background (Red): 255
25% Background (Red): 104.0
50% Background (Red): 124.0
75% Background (Red): 149.0

Mean Foreground (Red): 42.0582818407
Std Foreground (Red): 37.5876166819
Min Foreground (Red): 0
Max Foreground (Red): 255
25% Foreground (Red): 17.0
50% Foreground (Red): 24.0
75% Foreground (Red): 59.0

Mean Total (Green): 110.567739217
Std Total (Green): 41.1428633771
Min Total (Green): 0
Max Total (Green): 255
25% Total (Green): 93.0
50% Total (Green): 112.0
75% Total (Green): 138.0

Mean Background (Green): 111.988268651
Std Background (Green): 40.0328872798
Min Background (Green): 2
Max Background (Green): 255
25% Background (Green): 94.0

HBox(children=(IntProgress(value=0, max=350), HTML(value='')))


Dataset I_CA_01 loaded successfully
########## I_CA_01 ########## 

Mean Total (Red): 120.675688188
Std Total (Red): 36.3787183918
Min Total (Red): 1
Max Total (Red): 216
25% Total (Red): 97.0
50% Total (Red): 117.0
75% Total (Red): 155.0

Mean Background (Red): 122.542350466
Std Background (Red): 32.9822880638
Min Background (Red): 1
Max Background (Red): 216
25% Background (Red): 99.0
50% Background (Red): 118.0
75% Background (Red): 155.0

Mean Foreground (Red): 92.3677302529
Std Foreground (Red): 66.9678974411
Min Foreground (Red): 5
Max Foreground (Red): 214
25% Foreground (Red): 27.0
50% Foreground (Red): 75.0
75% Foreground (Red): 163.0

Mean Total (Green): 99.8076669598
Std Total (Green): 49.839844992
Min Total (Green): 0
Max Total (Green): 215
25% Total (Green): 64.0
50% Total (Green): 104.0
75% Total (Green): 143.0

Mean Background (Green): 104.125779105
Std Background (Green): 47.98143963
Min Background (Green): 0
Max Background (Green): 215
25% Background (Green): 80.0
50%

...


Dataset I_OC_01 loaded successfully
########## I_OC_01 ########## 

Mean Total (Red): 105.932490925
Std Total (Red): 42.5121344946
Min Total (Red): 0
Max Total (Red): 255
25% Total (Red): 81.0
50% Total (Red): 110.0
75% Total (Red): 130.0

Mean Background (Red): 106.178744915
Std Background (Red): 42.4055899336
Min Background (Red): 0
Max Background (Red): 255
25% Background (Red): 81.0
50% Background (Red): 110.0
75% Background (Red): 130.0

Mean Foreground (Red): 69.9850931939
Std Foreground (Red): 45.7657282168
Min Foreground (Red): 0
Max Foreground (Red): 255
25% Foreground (Red): 23.0
50% Foreground (Red): 73.0
75% Foreground (Red): 105.0

Mean Total (Green): 107.248047783
Std Total (Green): 43.0684130294
Min Total (Green): 1
Max Total (Green): 255
25% Total (Green): 82.0
50% Total (Green): 115.0
75% Total (Green): 132.0

Mean Background (Green): 107.728180947
Std Background (Green): 42.7732392245
Min Background (Green): 1
Max Background (Green): 255
25% Background (Green): 83.0


HBox(children=(IntProgress(value=0, max=300), HTML(value='')))


Dataset I_SI_01 loaded successfully
########## I_SI_01 ########## 

Mean Total (Red): 102.296203605
Std Total (Red): 65.3863195971
Min Total (Red): 6
Max Total (Red): 255
25% Total (Red): 33.0
50% Total (Red): 122.0
75% Total (Red): 161.0

Mean Background (Red): 103.07231613
Std Background (Red): 65.6114644966
Min Background (Red): 10
Max Background (Red): 255
25% Background (Red): 33.0
50% Background (Red): 126.0
75% Background (Red): 162.0

Mean Foreground (Red): 86.1868434652
Std Foreground (Red): 60.1962367657
Min Foreground (Red): 6
Max Foreground (Red): 255
25% Foreground (Red): 35.0
50% Foreground (Red): 61.0
75% Foreground (Red): 140.0

Mean Total (Green): 100.476680411
Std Total (Green): 63.3150846223
Min Total (Green): 5
Max Total (Green): 255
25% Total (Green): 35.0
50% Total (Green): 111.0
75% Total (Green): 158.0

Mean Background (Green): 101.267610888
Std Background (Green): 63.5729792347
Min Background (Green): 9
Max Background (Green): 255
25% Background (Green): 35.0


HBox(children=(IntProgress(value=0, max=300), HTML(value='')))


Dataset I_SM_01 loaded successfully
########## I_SM_01 ########## 

Mean Total (Red): 98.4174992437
Std Total (Red): 65.3772621032
Min Total (Red): 6
Max Total (Red): 255
25% Total (Red): 35.0
50% Total (Red): 73.0
75% Total (Red): 165.0

Mean Background (Red): 99.6040885687
Std Background (Red): 65.8130519577
Min Background (Red): 6
Max Background (Red): 255
25% Background (Red): 35.0
50% Background (Red): 76.0
75% Background (Red): 166.0

Mean Foreground (Red): 83.1284600709
Std Foreground (Red): 58.7478999882
Min Foreground (Red): 6
Max Foreground (Red): 255
25% Foreground (Red): 34.0
50% Foreground (Red): 60.0
75% Foreground (Red): 134.0

Mean Total (Green): 97.3340688131
Std Total (Green): 63.165817767
Min Total (Green): 5
Max Total (Green): 255
25% Total (Green): 37.0
50% Total (Green): 74.0
75% Total (Green): 160.0

Mean Background (Green): 98.6301663595
Std Background (Green): 63.6847319651
Min Background (Green): 7
Max Background (Green): 255
25% Background (Green): 37.0
50% 

## Pixel-Process histogram analysis

Above we analyzed the overall characteristics (mean/std) of all background / foreground pixel values for each scene.  
However, in fact, we don't care about the overall distribution, instead, the distribution per pixel-process is what we care about since it contains the information our classifier uses.

In [10]:
def pixelprocess_analysis(db_name):
    
    total_red = [[[] for _ in range(352)]  for _ in range(288)] 
    total_green = [[[] for _ in range(352)]  for _ in range(288)] 
    total_blue = [[[] for _ in range(352)]  for _ in range(288)] 
    
    background_red = [[[] for _ in range(352)]  for _ in range(288)] 
    background_green = [[[] for _ in range(352)]  for _ in range(288)] 
    background_blue = [[[] for _ in range(352)]  for _ in range(288)] 
    
    foreground_red = [[[] for _ in range(352)]  for _ in range(288)] 
    foreground_green = [[[] for _ in range(352)]  for _ in range(288)] 
    foreground_blue = [[[] for _ in range(352)]  for _ in range(288)] 
    
    stats = [[dict() for _ in range(352)]  for _ in range(288)] 
    
    data_loader = LASIESTADataLoader('../datasets/'+db_name, '../datasets/'+db_name+'-GT', db_name)
    
    print("Start loading dataset " + db_name)
    
    data_loader.set_next_frame_idx(0)

    for original_frame, label_frame in tqdm_notebook(data_loader, total=data_loader.get_nr_of_frames()):
        for i, row in enumerate(original_frame):
            for j, rgb in enumerate(row):
                total_red[i][j].append(rgb[0])
                total_green[i][j].append(rgb[1])
                total_blue[i][j].append(rgb[2])
          
                # Is background pixel
                if np.array_equal(label_frame[i,j], [0,0,0]):
                    background_red[i][j].append(original_frame[i,j,0])
                    background_green[i][j].append(original_frame[i,j,1])
                    background_blue[i][j].append(original_frame[i,j,2])  
                    
                # Is foreground pixel
                if np.array_equal(label_frame[i,j], [0,0,0]) is False and np.array_equal(label_frame[i,j], [128,128,128]) is False:
                    foreground_red[i][j].append(original_frame[i,j,0])
                    foreground_green[i][j].append(original_frame[i,j,1])
                    foreground_blue[i][j].append(original_frame[i,j,2])  

    for i in range(288):
        for j in range(352):
            stats[i][j]['total_mean_red'] = np.mean(total_red[i][j])
            stats[i][j]['total_mean_green'] = np.mean(total_green[i][j])
            stats[i][j]['total_mean_blue'] = np.mean(total_blue[i][j])

            stats[i][j]['total_std_red'] = np.std(total_red[i][j])
            stats[i][j]['total_std_green'] = np.std(total_green[i][j])
            stats[i][j]['total_std_blue'] = np.std(total_blue[i][j])

            stats[i][j]['total_percentile_25_red'] = np.percentile(total_red[i][j], 25)
            stats[i][j]['total_percentile_25_green'] = np.percentile(total_green[i][j], 25)
            stats[i][j]['total_percentile_25_blue'] = np.percentile(total_blue[i][j], 25)

            stats[i][j]['total_percentile_50_red'] = np.percentile(total_red[i][j], 50)
            stats[i][j]['total_percentile_50_green'] = np.percentile(total_green[i][j], 50)
            stats[i][j]['total_percentile_50_blue'] = np.percentile(total_blue[i][j], 50)

            stats[i][j]['total_percentile_75_red'] = np.percentile(total_red[i][j], 75)
            stats[i][j]['total_percentile_75_green'] = np.percentile(total_green[i][j], 75)
            stats[i][j]['total_percentile_75_blue'] = np.percentile(total_blue[i][j], 75)

            stats[i][j]['background_mean_red'] = np.mean(background_red[i][j])
            stats[i][j]['background_mean_green'] = np.mean(background_green[i][j])
            stats[i][j]['background_mean_blue'] = np.mean(background_blue[i][j])

            stats[i][j]['background_std_red'] = np.std(background_red[i][j])
            stats[i][j]['background_std_green'] = np.std(background_green[i][j])
            stats[i][j]['background_std_blue'] = np.std(background_blue[i][j])

            if len(background_red[i][j]) != 0:
                stats[i][j]['background_percentile_25_red'] = np.percentile(background_red[i][j], 25)
            else:
                stats[i][j]['background_percentile_25_red'] = '-'
                
            if len(background_green[i][j]) != 0:
                stats[i][j]['background_percentile_25_green'] = np.percentile(background_green[i][j], 25)
            else:
                stats[i][j]['background_percentile_25_green'] = '-'
                
            if len(background_blue[i][j]) != 0:
                stats[i][j]['background_percentile_25_blue'] = np.percentile(background_blue[i][j], 25)
            else:
                stats[i][j]['background_percentile_25_blue'] = '-'

            if len(background_red[i][j]) != 0:
                stats[i][j]['background_percentile_50_red'] = np.percentile(background_red[i][j], 50)
            else:
                stats[i][j]['background_percentile_50_red'] = '-'
                
            if len(background_green[i][j]) != 0:
                stats[i][j]['background_percentile_50_green'] = np.percentile(background_green[i][j], 50)
            else:
                stats[i][j]['background_percentile_50_green'] = '-'
                
            if len(background_blue[i][j]) != 0:
                stats[i][j]['background_percentile_50_blue'] = np.percentile(background_blue[i][j], 50)
            else:
                stats[i][j]['background_percentile_50_blue'] = '-'

            if len(background_red[i][j]) != 0:
                stats[i][j]['background_percentile_75_red'] = np.percentile(background_red[i][j], 75)
            else:
                stats[i][j]['background_percentile_75_red'] = '-'
                
            if len(background_green[i][j]) != 0:
                stats[i][j]['background_percentile_75_green'] = np.percentile(background_green[i][j], 75)
            else:
                stats[i][j]['background_percentile_75_green'] = '-'
                
            if len(background_blue[i][j]) != 0:
                stats[i][j]['background_percentile_75_blue'] = np.percentile(background_blue[i][j], 75)
            else:
                stats[i][j]['background_percentile_75_blue'] = '-'

            stats[i][j]['foreground_mean_red'] = np.mean(foreground_red[i][j])
            stats[i][j]['foreground_mean_green'] = np.mean(foreground_green[i][j])
            stats[i][j]['foreground_mean_blue'] = np.mean(foreground_blue[i][j])

            stats[i][j]['foreground_std_red'] = np.std(foreground_red[i][j])
            stats[i][j]['foreground_std_green'] = np.std(foreground_green[i][j])
            stats[i][j]['foreground_std_blue'] = np.std(foreground_blue[i][j])

            if len(foreground_red[i][j]) != 0:
                stats[i][j]['foreground_percentile_25_red'] = np.percentile(foreground_red[i][j], 25)
            else:
                stats[i][j]['foreground_percentile_25_red'] = '-'
                
            if len(foreground_green[i][j]) != 0:
                stats[i][j]['foreground_percentile_25_green'] = np.percentile(foreground_green[i][j], 25)
            else:
                stats[i][j]['foreground_percentile_25_green']  = '-'
                
            if len(foreground_blue[i][j]) != 0:
                stats[i][j]['foreground_percentile_25_blue'] = np.percentile(foreground_blue[i][j], 25)
            else:
                stats[i][j]['foreground_percentile_25_blue'] = '-'

            if len(foreground_red[i][j]) != 0:
                stats[i][j]['foreground_percentile_50_red'] = np.percentile(foreground_red[i][j], 50)
            else:
                stats[i][j]['foreground_percentile_50_red'] = '-'
                
            if len(foreground_green[i][j]) != 0:
                stats[i][j]['foreground_percentile_50_green'] = np.percentile(foreground_green[i][j], 50)
            else:
                stats[i][j]['foreground_percentile_50_green'] = '-'
                
            if len(foreground_blue[i][j]) != 0:
                stats[i][j]['foreground_percentile_50_blue'] = np.percentile(foreground_blue[i][j], 50)
            else:
                stats[i][j]['foreground_percentile_50_blue'] = '-'
                
            if len(foreground_red[i][j]) != 0:
                stats[i][j]['foreground_percentile_75_red'] = np.percentile(foreground_red[i][j], 75)
            else:
                stats[i][j]['foreground_percentile_75_red'] = '-'
                
            if len(foreground_green[i][j]) != 0:
                stats[i][j]['foreground_percentile_75_green'] = np.percentile(foreground_green[i][j], 75)
            else:
                stats[i][j]['foreground_percentile_75_green'] = '-'
                
            if len(foreground_blue[i][j]) != 0:
                stats[i][j]['foreground_percentile_75_blue'] = np.percentile(foreground_blue[i][j], 75)
            else:
                stats[i][j]['foreground_percentile_75_blue'] = '-'
                
    print("Dataset " + db_name + ' loaded successfully')
    
    return stats
        

In [ ]:
def stats_for_pixel(stats,x,y):

    print('Mean Total (Red): ' + str(stats[y][x]['total_mean_red']))
    print('Std Total (Red): ' + str(stats[y][x]['total_std_red']))
    print('25% Total (Red): ' + str(stats[y][x]['total_percentile_25_red']))
    print('50% Total (Red): ' + str(stats[y][x]['total_percentile_50_red']))
    print('75% Total (Red): ' + str(stats[y][x]['total_percentile_75_red']))
    print()
    print('Mean Foreground (Red): ' + str(stats[y][x]['foreground_mean_red']))
    print('Std Foreground (Red): ' + str(stats[y][x]['foreground_std_red']))
    print('25% Foreground (Red): ' + str(stats[y][x]['foreground_percentile_25_red']))
    print('50% Foreground (Red): ' + str(stats[y][x]['foreground_percentile_50_red']))
    print('75% Foreground (Red): ' + str(stats[y][x]['foreground_percentile_75_red']))
    print()
    print('Mean Background (Red): ' + str(stats[y][x]['background_mean_red']))
    print('Std Background (Red): ' + str(stats[y][x]['background_std_red']))
    print('25% Background (Red): ' + str(stats[y][x]['background_percentile_25_red']))
    print('50% Background (Red): ' + str(stats[y][x]['background_percentile_50_red']))
    print('75% Background (Red): ' + str(stats[y][x]['background_percentile_75_red']))
    print()
    print('Mean Total (Green): ' + str(stats[y][x]['total_mean_green']))
    print('Std Total (Green): ' + str(stats[y][x]['total_std_green']))
    print('25% Total (Green): ' + str(stats[y][x]['total_percentile_25_green']))
    print('50% Total (Green): ' + str(stats[y][x]['total_percentile_50_green']))
    print('75% Total (Green): ' + str(stats[y][x]['total_percentile_75_green']))
    print()
    print('Mean Foreground (Green): ' + str(stats[y][x]['foreground_mean_green']))
    print('Std Foreground (Green): ' + str(stats[y][x]['foreground_std_green']))
    print('25% Foreground (Green): ' + str(stats[y][x]['foreground_percentile_25_green']))
    print('50% Foreground (Green): ' + str(stats[y][x]['foreground_percentile_50_green']))
    print('75% Foreground (Green): ' + str(stats[y][x]['foreground_percentile_75_green']))
    print()
    print('Mean Background (Green): ' + str(stats[y][x]['background_mean_green']))
    print('Std Background (Green): ' + str(stats[y][x]['background_std_green']))
    print('25% Background (Green): ' + str(stats[y][x]['background_percentile_25_green']))
    print('50% Background (Green): ' + str(stats[y][x]['background_percentile_50_green']))
    print('75% Background (Green): ' + str(stats[y][x]['background_percentile_75_green']))
    print()
    print('Mean Total (Blue): ' + str(stats[y][x]['total_mean_blue']))
    print('Std Total (Blue): ' + str(stats[y][x]['total_std_blue']))
    print('25% Total (Blue): ' + str(stats[y][x]['total_percentile_25_blue']))
    print('50% Total (Blue): ' + str(stats[y][x]['total_percentile_50_blue']))
    print('75% Total (Blue): ' + str(stats[y][x]['total_percentile_75_blue']))
    print()
    print('Mean Foreground (Blue): ' + str(stats[y][x]['foreground_mean_blue']))
    print('Std Foreground (Blue): ' + str(stats[y][x]['foreground_std_blue']))
    print('25% Foreground (Blue): ' + str(stats[y][x]['foreground_percentile_25_blue']))
    print('50% Foreground (Blue): ' + str(stats[y][x]['foreground_percentile_50_blue']))
    print('75% Foreground (Blue): ' + str(stats[y][x]['foreground_percentile_75_blue']))
    print()
    print('Mean Background (Blue): ' + str(stats[y][x]['background_mean_blue']))
    print('Std Background (Blue): ' + str(stats[y][x]['background_std_blue']))
    print('25% Background (Blue): ' + str(stats[y][x]['background_percentile_25_blue']))
    print('50% Background (Blue): ' + str(stats[y][x]['background_percentile_50_blue']))
    print('75% Background (Blue): ' + str(stats[y][x]['background_percentile_75_blue']))
    print()

### Scene: I_SI_01

In [11]:
i_si_01_stats = pixelprocess_analysis('I_SI_01')

Start loading dataset I_SI_01


HBox(children=(IntProgress(value=0, max=300), HTML(value='')))

/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Dataset I_SI_01 loaded successfully


In [23]:
stats_for_pixel(i_si_01_stats, 240, 160)

Mean Total (Red): 186.013333333
Std Total (Red): 24.1562929459
25% Total (Red): 188.75
50% Total (Red): 191.0
75% Total (Red): 193.0

Mean Foreground (Red): 113.894736842
Std Foreground (Red): 56.4324418984
25% Foreground (Red): 76.5
50% Foreground (Red): 128.0
75% Foreground (Red): 163.5

Mean Background (Red): 191.208633094
Std Background (Red): 4.24344576312
25% Background (Red): 189.0
50% Background (Red): 191.0
75% Background (Red): 193.75

Mean Total (Green): 184.966666667
Std Total (Green): 27.3379874087
25% Total (Green): 188.0
50% Total (Green): 191.0
75% Total (Green): 193.0

Mean Foreground (Green): 98.4736842105
Std Foreground (Green): 56.8112001754
25% Foreground (Green): 51.0
50% Foreground (Green): 91.0
75% Foreground (Green): 147.0

Mean Background (Green): 191.21942446
Std Background (Green): 4.26868095304
25% Background (Green): 189.0
50% Background (Green): 191.0
75% Background (Green): 193.0

Mean Total (Blue): 183.696666667
Std Total (Blue): 28.6416594414
25% Total

### Scene: I_BS_01

In [27]:
i_bs_01_stats = pixelprocess_analysis('I_BS_01')

Start loading dataset I_BS_01


HBox(children=(IntProgress(value=0, max=275), HTML(value='')))

/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Dataset I_BS_01 loaded successfully


In [28]:
stats_for_pixel(i_bs_01_stats, 170, 150)

Mean Total (Red): 101.556363636
Std Total (Red): 47.4171383042
25% Total (Red): 58.5
50% Total (Red): 132.0
75% Total (Red): 134.0

Mean Foreground (Red): 35.686746988
Std Foreground (Red): 26.4032344271
25% Foreground (Red): 18.0
50% Foreground (Red): 21.0
75% Foreground (Red): 50.5

Mean Background (Red): 133.386904762
Std Background (Red): 2.18470221148
25% Background (Red): 132.0
50% Background (Red): 133.0
75% Background (Red): 134.0

Mean Total (Green): 96.8327272727
Std Total (Green): 40.4591064232
25% Total (Green): 57.5
50% Total (Green): 124.0
75% Total (Green): 126.0

Mean Foreground (Green): 39.8674698795
Std Foreground (Green): 16.0066033695
25% Foreground (Green): 28.0
50% Foreground (Green): 32.0
75% Foreground (Green): 54.0

Mean Background (Green): 125.333333333
Std Background (Green): 2.12038481084
25% Background (Green): 124.0
50% Background (Green): 125.0
75% Background (Green): 126.0

Mean Total (Blue): 71.9781818182
Std Total (Blue): 36.2090831945
25% Total (Blue)

### Scene: I_IL_01

In [29]:
i_il_01_stats = pixelprocess_analysis('I_IL_01')

Start loading dataset I_IL_01


HBox(children=(IntProgress(value=0, max=300), HTML(value='')))

/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Dataset I_IL_01 loaded successfully


In [30]:
stats_for_pixel(i_il_01_stats, 150, 150)

Mean Total (Red): 36.96
Std Total (Red): 18.8004893553
25% Total (Red): 31.0
50% Total (Red): 33.0
75% Total (Red): 37.0

Mean Foreground (Red): 133.75
Std Foreground (Red): 48.4838890767
25% Foreground (Red): 102.25
50% Foreground (Red): 131.0
75% Foreground (Red): 178.5

Mean Background (Red): 34.0412371134
Std Background (Red): 3.78798296781
25% Background (Red): 31.0
50% Background (Red): 33.0
75% Background (Red): 37.0

Mean Total (Green): 38.55
Std Total (Green): 13.2710524576
25% Total (Green): 34.0
50% Total (Green): 37.0
75% Total (Green): 40.0

Mean Foreground (Green): 95.875
Std Foreground (Green): 50.0060933787
25% Foreground (Green): 58.5
50% Foreground (Green): 74.0
75% Foreground (Green): 147.5

Mean Background (Green): 36.8487972509
Std Background (Green): 3.87579814595
25% Background (Green): 34.0
50% Background (Green): 37.0
75% Background (Green): 39.0

Mean Total (Blue): 37.85
Std Total (Blue): 10.2460154857
25% Total (Blue): 33.0
50% Total (Blue): 37.0
75% Total (B